# Amazon Scraper
## ADS-509 Final
### Ivan Chavez, Uyen Pham, Lane Whitmore

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import random
import re
import itertools
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import collections


In [4]:
# top 50 urls for movies, books and video games
m_url = 'https://www.amazon.com/Best-Sellers-Movies-TV-Prime-Video/zgbs/movies-tv/2958933011/ref=zg_bs_nav_movies-tv_1'
b_url = 'https://www.amazon.com/gp/bestsellers/books/ref=s9_acss_bw_cg_bsmpill_1c1_w?ref=bsm_nav_pill_print&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-1&pf_rd_r=56RBBA17NCWBRFTBZVMP&pf_rd_t=101&pf_rd_p=ef8cebb8-ad4b-453c-8030-a931d3822444&pf_rd_i=16857165011'
vg_url = 'https://www.amazon.com/Best-Sellers-Video-Games-Xbox-Series-X-S-Games/zgbs/videogames/20972814011/ref=zg_bs_nav_videogames_2_20972798011'

In [ ]:
'_cDEzb_p13n-sc-css-line-clamp-2_EWgCb'

In [13]:
# first the top 50 urls need to be scraped to grab the url for each product's description and review page
def scrape(url):
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    driver.get(url)

    time.sleep(8)
# refresh to get past amazon block
    driver.refresh()

# scroll page to find all 50
    driver.execute_script("window.scrollTo(0, 10000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 14000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 15000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 16000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 17000)") 


# lists to save the data to
    hrefs = []
    r_hrefs = []
    titles = []

    # grabbing the title of each product
    title = driver.find_elements_by_css_selector("div[class^='_cDEzb_p13n-sc-css-line-clamp-']".format())
    for t in title:
        if int(t.get_attribute('class').split('-')[-1].split('_')[0]) < 2:
            continue
        else:
            titles.append(t.text)
    
    # getting all elements that contain product href
    m_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@role='link']")
    
    # all elements containing 
    mr_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@title]")

    # loop through the elements to extract the href for both 
    for i in m_list:
        href1 = i.get_attribute("href")
        hrefs.append(href1)

    for m in mr_list:
        hrefm = m.get_attribute("href")
        r_hrefs.append(hrefm)
    
    # print lens for feedback
    print(len(titles))
    print(len(r_hrefs))
    
        
    time.sleep(10)
    
    # Removing the extra hrefs
    hrefs = list(dict.fromkeys(hrefs))
    print(len(hrefs))

    #fin_titles = titles[::2]
    #print(len(fin_titles)
    
    
    # I ended up prefering manually closing this one to see which products were missing reviews and/or were duplicates on amazon's side
    #driver.close()

    return hrefs, r_hrefs, titles

In [14]:
m_hrefs, mr_hrefs, m_titles = scrape(m_url)

50
46
50


In [48]:
tmp = pd.DataFrame({'title': m_titles, 'hrefs': m_hrefs})
m_df = pd.concat([tmp[~tmp['title'].isin(['Hypnotic', 'Straight No Chaser', 'Book Club: The Next Chapter'])].reset_index(drop = True), pd.DataFrame({'rhrefs': mr_hrefs})], axis = 1)
m_df.drop_duplicates(subset='title', inplace=True)

In [50]:
print(len(m_df))
m_df

38


,title,hrefs,rhrefs
0,The Super Mario Bros. Movie,https://www.amazon.com/Super-Mario-Bros-Movie/...,https://www.amazon.com/product-reviews/B0BXDGV...
1,John Wick: Chapter 4 - Bonus X-Ray Edition,https://www.amazon.com/John-Wick-Chapter-Bonus...,https://www.amazon.com/product-reviews/B0BX9MZ...
2,John Wick: Chapter 4,https://www.amazon.com/placeholder_title-John-...,https://www.amazon.com/product-reviews/B0B6QNS...
3,65,https://www.amazon.com/65-Adam-Driver/dp/B0B8T...,https://www.amazon.com/product-reviews/B0B8TVD...
4,Dungeons & Dragons: Honor Among Thieves,https://www.amazon.com/Dungeons-Dragons-Honor-...,https://www.amazon.com/product-reviews/B0B693Q...
6,Spider-Man: Into the Spider-Verse,https://www.amazon.com/Spider-Man-Into-Spider-...,https://www.amazon.com/product-reviews/B07L8Q3...
7,Creed III,https://www.amazon.com/Creed-III-Michael-B-Jor...,https://www.amazon.com/product-reviews/B0B8TFG...
8,Guy Ritchie's The Covenant,https://www.amazon.com/Guy-Ritchies-Covenant-J...,https://www.amazon.com/product-reviews/B0B8P5C...
9,Avatar: The Way of Water,https://www.amazon.com/Avatar-Way-Water-Sam-Wo...,https://www.amazon.com/product-reviews/B0B72TV...
10,Cocaine Bear,https://www.amazon.com/Cocaine-Bear-Keri-Russe...,https://www.amazon.com/product-reviews/B0B6JNB...


#### Very important, the products without reviews need to be manually removed here sadly. 

In [ ]:
m_asin = [url.split('/dp/')[1].split('/')[0] for url in m_hrefs]
mr_asin = [url.split('https://www.amazon.com/product-reviews/')[1].split('/')[0] for url in mr_hrefs]

#get the no review links
urls_not_in_mr = [url for url in m_hrefs if url.split('/dp/')[1].split('/')[0] in m_asin_not_in_mr]
print(urls_not_in_mr)

In [17]:
#filter out the no-review links
m_hrefs = [url for url in m_hrefs if url.split('/dp/')[1].split('/')[0] not in m_asin_not_in_mr]
print(len(m_hrefs))

0


In [12]:
# Find and print the duplicates
names = [url.split('https://www.amazon.com/')[1].split('/')[0] for url in m_hrefs]
duplicates = [name for name, count in collections.Counter(names).items() if count > 1]
for duplicate in duplicates:
    print(f"Duplicate title: {duplicate}")
    indexes = [index for index, name in enumerate(names) if name == duplicate]
    for index in indexes:
        print(f"URL: {m_hrefs[index]}, Index: {index}")
    print()

Duplicate title: Super-Mario-Bros-Movie
URL: https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BXDGVMHS/ref=zg_bs_2958933011_sccl_1/132-2453380-0474863?psc=1, Index: 0
URL: https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BYL153QV/ref=zg_bs_2958933011_sccl_6/132-2453380-0474863?psc=1, Index: 5

Duplicate title: 65-Adam-Driver
URL: https://www.amazon.com/65-Adam-Driver/dp/B0B8TVD86K/ref=zg_bs_2958933011_sccl_4/132-2453380-0474863?psc=1, Index: 3
URL: https://www.amazon.com/65-Adam-Driver/dp/B0B8TVG6WW/ref=zg_bs_2958933011_sccl_37/132-2453380-0474863?psc=1, Index: 36

Duplicate title: Dungeons-Dragons-Honor-Among-Thieves
URL: https://www.amazon.com/Dungeons-Dragons-Honor-Among-Thieves/dp/B0B693QYTD/ref=zg_bs_2958933011_sccl_5/132-2453380-0474863?psc=1, Index: 4
URL: https://www.amazon.com/Dungeons-Dragons-Honor-Among-Thieves/dp/B0B6957VTZ/ref=zg_bs_2958933011_sccl_40/132-2453380-0474863?psc=1, Index: 39

Duplicate title: Spider-Man-Into-Spider-Verse-4K-UHD
URL: https://www.amazon.com/

In [25]:
m_df = pd.DataFrame({'title': m_titles, 'phref': m_refs, 'rhrefs': mr_hrefs}).drop_duplicates(subset="title")

For some reason Amazon will list the same movie twice and list movies without reviews. This led to this manual control over the dataframe after scraping. 

## Pull book URLs

In [ ]:
b_prod, b_review = scrape(b_url)

## Pull Video Game URLs

In [ ]:
v_prod, v_review = scrape(vg_url)

## Scrape Movies' reviews and details

In [51]:
def scrape_movie_details(list_urls):

    # lists for movie details
    title = []
    imdb = []
    amazon_rating = []
    review_count = []
    length = []
    year = []
    rating = []
    price = []


    # activate driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()

        title.append(driver.find_element_by_xpath("//h1[@data-automation-id='title']").text)
        # some products that are new do not have official ratings and a bypass needs to be created. Used the NoSuchElement exception from selenium to bypass this

        # imbd rating
        try:
            im = driver.find_element_by_xpath("//span[@data-automation-id='imdb-rating-badge']").text
            imdb.append(im.split(" ")[1])
        except NoSuchElementException:
            print("No IMDB Rating")
            continue

        
        # amazon rating
        try:
            ar = driver.find_element_by_xpath("//span[@class='_3mK_sl _3NWEVD']").get_attribute("aria-label")
            amazon_rating.append(ar.split(" ")[1])
        except NoSuchElementException:
            print("No Amazon Rating")
            amazon_rating.append(np.nan)
        
        # amount of amazon reviews
        try:
            review_count.append(driver.find_element_by_xpath("//span[@class='_1Y_bt3']").text)
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        # movie runtime
        try:  
            length.append(driver.find_element_by_xpath("//span[@data-automation-id='runtime-badge']").text)
        except NoSuchElementException:
            print("No Amazon Runtime")
            length.append(np.nan)
        
        # year released
        year.append(driver.find_element_by_xpath("//span[@data-automation-id='release-year-badge']").text)

        # age rating G|PG etc..
        rating.append(driver.find_element_by_xpath("//span[@class='G8xF_x']").text)
        
        # price in whole dollar
        try:
            pr = driver.find_element_by_xpath("//button[@class='_2X_Irl tvod-button av-button']").text
            price.append(re.findall("\d+", pr)[0])
        except NoSuchElementException:
            print("No Listed Price")
            price.append(np.nan)

        time.sleep(random.randint(8,15))
    
    # close driver
    driver.close()   

    # create dataframe
    data = pd.DataFrame({"title": title, "imbd_rating": imdb,"amazon_rating":amazon_rating, "review_count": review_count, "runtime": length, 
                         "year": year, "rating": rating, "price": price})
    
    return data


        

def scrape_reviews(list_urls):

    # open driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:
        # lists for reviews
        rating = []
        rating_fin = []
        review = [] 
        name = []
        title = []
        date_place = [] 
        movie = []

        # loop through range of pages | stopped at 900(or 9,000 reviews) to save time and memory
        for n in range(1, 900):

            # using the url plus this handy snippet to loop through all review pages
            driver.get(i+f"?ie=UTF8&pageNumber={n}")
            
            time.sleep(random.randint(5,12))
            driver.refresh()

            # grabbing names past first two that are always there to decide if page has reviews
            names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]
            rev_check = []
            for a in names:
                if a.text == '':
                    continue
                else:
                    rev_check.append(a.text)

            # if page has 1  or more name (reviews) scrape info below
            if len(rev_check) > 0:
                
                # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]

                # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                # rating
                ratings = driver.find_elements_by_xpath("//a[@class='a-link-normal']")

                # review title
                rev_title = driver.find_elements_by_xpath("//a[@class='a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold']")

                # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                # movie name
                movies = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)

                for na in names:
                    if na.text == '':
                        continue
                    else:
                        name.append(na.text)

                for rat in ratings:
                    rating.append(rat.get_attribute("title"))

                            
                for t in rev_title:
                    title.append(t.text)
                            
                for d in dateplaces:
                    date_place.append(d.text)
                    
                for m in movies:
                    for mo in [m.text] * len(rev_check):
                        movie.append(mo)

                # give amazon a little break     
                time.sleep(random.randint(8,15))

            # if the amount of names is equal to 0, break to next movie as there are no more reviews
            elif len(rev_check) == 0:
                print("There are no more reviews for this product!")
                break
    
        # cleaning up rating to remove some unwanted elements that were scraped
        for rat in rating:
            if rat != '':
                if rat.split()[1] == 'out':
                    rating_fin.append(rat.split()[0])



        # see lens to find out if there were any mistakes
        print(len(name))
        print(len(review))
        print(len(rating_fin))
        print(len(title))
        print(len(date_place))
        print(len(movie))

        savefile = str(str(movie[0]).replace(" ", "") + "_review_df")

        print(name)
        # create review dataframe
        data = pd.DataFrame({"name": name, "movie_title": movie, "review_title": title, "review": review, "rating": rating_fin, "date_place": date_place})
        data.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/movies/{}.csv'.format(savefile))

    # close driver
    driver.close()

In [54]:
movie_details_df = scrape_movie_details(m_df['hrefs'])

In [ ]:
movie_details_df.sort_values('title', ascending=False)          

,title,imbd_rating,amazon_rating,review_count,runtime,year,rating,price
17,The Whale,7.7,4.3,"10,881",1 h 56 min,2022,18+,5
0,The Super Mario Bros. Movie,7.2,4.8,"1,675",1 h 32 min,2023,PG,24
26,The Ritual Killer,4.5,3.7,"2,290",1 h 31 min,2023,16+,4
22,The Pope's Exorcist,6.1,4.3,934,1 h 43 min,2023,R,19
23,The Fast and the Furious,6.8,4.7,"1,214",1 h 47 min,2001,PG-13,3
32,The Croods: A New Age,6.9,4.8,"59,776",1 h 35 min,2020,PG,3
7,Spider-Man: Into the Spider-Verse,8.4,4.8,"57,076",1 h 56 min,2018,PG,4
12,Sisu,7.0,4.5,210,1 h 31 min,2023,R,19
24,Shazam! Fury of the Gods,6.0,4.3,232,2 h 10 min,2023,PG-13,5
16,Scream VI,6.6,4.3,392,2 h 2 min,2023,18+,5


In [ ]:
movie_details_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/md_df.csv')

In [ ]:
scrape_reviews(m_df['rhrefs'])

There are no more reviews for this product!
202
202
202
202
202
202
There are no more reviews for this product!
162
162
162
162
162
162
There are no more reviews for this product!
162
162
162
162
162
162
There are no more reviews for this product!
373
373
373
373
373
373
There are no more reviews for this product!
323
325
325
325
325
323


ValueError: All arrays must be of the same length

## Scraping Books

In [ ]:
def scrape_book_details(list_urls):
    book = []
    amazon_rating = []
    review_count = []
    length = []
    year = []
    rating = []
    price = []
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()
        book.append(driver.find_element_by_xpath("//h1[@data-automation-id='title']").text)

        # some products that are new do not have official ratings and a bypass needs to be created
        try:
            ar = driver.find_element_by_xpath("//span[@class='a-icon-alt']").text
            amazon_rating.append(ar.split(" ")[1])
        except NoSuchElementException:
            print("No Amazon Rating")
            amazon_rating.append(np.nan)
        
        try:
            review_count.append(driver.find_element_by_xpath("//span[@class='_1Y_bt3']").text)
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        try:  
            length.append(driver.find_element_by_xpath("//div[@class='a-section a-spacing-none a-text-center rpi-attribute-value']").text)
        except NoSuchElementException:
            print("No Amazon Page Length")
            length.append(np.nan)
            
        year.append(driver.find_element_by_xpath("//span[@id='productSubtitle']").text.split()[-1])

        rating.append(driver.find_element_by_xpath("//a[@class='a-link-normal'][@href='#']").text)
        
        try:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[1].text)
        except NoSuchElementException:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[0].text)

        time.sleep(random.randint(8,15))
    
    driver.close()   
    data = pd.DataFrame({"title": book,"amazon_rating":amazon_rating, "review_count": review_count, "runtime": length, 
                         "year": year, "rating": rating, "price": price})
    return data

    

In [ ]:
def scrape_b_reviews(list_urls):

    # lists for reviews
    rating = []
    rating_fin = []
    review = [] 
    name = []
    title = []
    date_place = [] 
    book = []

    # open driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:

        # loop through range of pages | stopped at 900(or 9,000 reviews) to save time and memory
        for n in range(1, 900):

            # using the url plus this handy snippet to loop through all review pages
            driver.get(i+f"?ie=UTF8&pageNumber={n}")
            
            time.sleep(random.randint(5,12))
            driver.refresh()

            # grabbing names past first two that are always there to decide if page has reviews
            names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]
            rev_check = []
            for a in names:
                rev_check.append(a.text)

            # if page has 1  or more name (reviews) scrape info below
            if len(rev_check) > 0:
                
                # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]

                # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                # rating
                ratings = driver.find_elements_by_xpath("//a[@class='a-link-normal']")

                # review title
                rev_title = driver.find_elements_by_xpath("//a[@class='a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold']")

                # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                # movie name
                books = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)

                for na in names:
                    name.append(na.text)

                for rat in ratings:
                    rating.append(rat.get_attribute("title"))

                            
                for t in rev_title:
                    title.append(t.text)
                            
                for d in dateplaces:
                    date_place.append(d.text)
                    
                for m in books:
                    for mo in [m.text] * len(rev_check):
                        book.append(mo)

                # give amazon a little break     
                time.sleep(random.randint(8,15))

            # if the amount of names is equal to 0, break to next movie as there are no more reviews
            elif len(rev_check) == 0:
                print("There are no more reviews for this product!")
                break
    
    # cleaning up rating to remove some unwanted elements that were scraped
    for rat in rating:
        if rat != '':
            if rat.split()[1] == 'out':
                rating_fin.append(rat.split()[0])


    # close driver
    driver.close()

    # see lens to find out if there were any mistakes
    print(len(name))
    print(len(review))
    print(len(rating_fin))
    print(len(title))
    print(len(date_place))
    print(len(book))

    # create review dataframe
    data = pd.DataFrame({"name": name, "book_title": book, "review_title": title, "review": review, "rating": rating_fin, "date_place": date_place})

    return data

## Scraping Video Games

In [ ]:
def scrape_games_details(list_urls):
    game = []
    amazon_rating = []
    review_count = []
    rating = []
    price = []
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()
        game.append(driver.find_element_by_xpath("//span[@id='productTitle']").text)

        # some products that are new do not have official ratings and a bypass needs to be created
        try:
            ar = driver.find_element_by_xpath("//span[@class='a-icon-alt']").text
            amazon_rating.append(ar.split(" ")[1])
        except NoSuchElementException:
            print("No Amazon Rating")
            amazon_rating.append(np.nan)
        
        try:
            review_count.append(driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']").text.split()[0])
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        rating.append(driver.find_element_by_xpath("//a[@class='a-popover-trigger a-declarative'][@role='button']").text)

        try:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[1].text)
        except NoSuchElementException:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[0].text)
        

        time.sleep(random.randint(8,15))
    
    driver.close()  

    data = pd.DataFrame({"title": game,"amazon_rating":amazon_rating, 
                         "review_count": review_count, "rating": rating, 
                         "price": price})
    
    return data

In [ ]:
def scrape_vg_reviews(list_urls):

    # lists for reviews
    rating = []
    rating_fin = []
    review = [] 
    name = []
    title = []
    date_place = [] 
    game = []

    # open driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:

        # loop through range of pages | stopped at 900(or 9,000 reviews) to save time and memory
        for n in range(1, 900):

            # using the url plus this handy snippet to loop through all review pages
            driver.get(i+f"?ie=UTF8&pageNumber={n}")
            
            time.sleep(random.randint(5,12))
            driver.refresh()

            # grabbing names past first two that are always there to decide if page has reviews
            names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]
            rev_check = []
            for a in names:
                rev_check.append(a.text)

            # if page has 1  or more name (reviews) scrape info below
            if len(rev_check) > 0:
                
                # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]

                # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                # rating
                ratings = driver.find_elements_by_xpath("//a[@class='a-link-normal']")

                # review title
                rev_title = driver.find_elements_by_xpath("//a[@class='a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold']")

                # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                # movie name
                games = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)

                for na in names:
                    name.append(na.text)

                for rat in ratings:
                    rating.append(rat.get_attribute("title"))

                            
                for t in rev_title:
                    title.append(t.text)
                            
                for d in dateplaces:
                    date_place.append(d.text)
                    
                for m in games:
                    for mo in [m.text] * len(rev_check):
                        game.append(mo)

                # give amazon a little break     
                time.sleep(random.randint(8,15))

            # if the amount of names is equal to 0, break to next movie as there are no more reviews
            elif len(rev_check) == 0:
                print("There are no more reviews for this product!")
                break
    
    # cleaning up rating to remove some unwanted elements that were scraped
    for rat in rating:
        if rat != '':
            if rat.split()[1] == 'out':
                rating_fin.append(rat.split()[0])


    # close driver
    driver.close()

    # see lens to find out if there were any mistakes
    print(len(name))
    print(len(review))
    print(len(rating_fin))
    print(len(title))
    print(len(date_place))
    print(len(game))

    # create review dataframe
    data = pd.DataFrame({"name": name, "game_title":  game, "review_title": title, "review": review, "rating": rating_fin, "date_place": date_place})

    return data